# Variational Autoencoder

- https://github.com/pytorch/examples/blob/master/vae/main.py
- http://cympfh.cc/paper/VAE.html
- https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
- https://qiita.com/kenmatsu4/items/b029d697e9995d93aa24

- zがN(0,1)になるように制約を加える
- zの次元は２次元にします。どうでしょう、訓練データのMNISTデータセットが2次元の正規分布に従う円の上に散らばっている様子が見て取れると思います。
- VAEはzが正規分布に従うように設計されており、正規分布に従う乱数を学習時に取り入れているので、この乱数によるブレによって似た形状のものを近くに寄せる効果があるためです。つまり、同じ画像を入力しても毎回ちょっとずれたところにzがプロットされ、そのzからDecoderによって生成する画像を入力画像と同じようにするためです。

- log-likelihoodの最大化 = binary cross entropyの最小化？
- https://www.quora.com/What-are-the-differences-between-maximum-likelihood-and-cross-entropy-as-a-loss-function

In [16]:
import os
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [17]:
batch_size = 128
epochs = 10
seed = 1
log_interval = 10
out_dir = './vae'

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')

In [18]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [19]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data',
                   train=True,
                   download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

In [20]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data',
                   train=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

In [21]:
print(len(train_loader))
print(len(test_loader))

469
79


In [37]:
class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)  # mu
        self.fc22 = nn.Linear(400, 20)  # logvar
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        x = x.view(-1, 784)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

- `exp_()` でなくて `exp()` でも同じでは？
- eps（N(0,1) の乱数）がベクトルになっている
- 潜在変数の各次元ごとに異なる乱数をかけることになる
    - 数式見るとスカラーみたいだけどこの実装でよいのか？
    - `std.data.new(1)` でも復元はできる => 比較してみる！
- `eps = Variable(std.data.new(1).normal_())` ではダメ？
- $logvar = \log \Sigma^2$
- $std = \exp ( 0.5 \log \Sigma^2) = \exp (\log \Sigma) = \Sigma$

In [38]:
model = VAE()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [39]:
data, _ = iter(train_loader).next()
print(data.size())

torch.Size([128, 1, 28, 28])


In [40]:
outputs, mu, logvar = model(Variable(data))
print(outputs.size())
print(mu.size())
print(logvar.size())

torch.Size([128, 784])
torch.Size([128, 20])
torch.Size([128, 20])


In [41]:
def loss_function(recon_x, x, mu, logvar):
    # size_average=Falseなのでバッチ内のサンプルの合計loss
    # reconstruction loss 入力画像をどのくらい正確に復元できたか？
    # 数式では対数尤度の最大化だが交差エントロピーlossの最小化と等価
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)

    # 潜在空間zに対する正則化項
    # P(z|x) が N(0, I)に近くなる（KL-distanceが小さくなる）ようにする
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [42]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        if cuda:
            data = Variable(data.cuda())
        else:
            data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
    
    print('Epoch: {} Average loss: {:.4f}'.format(
        epoch,
        train_loss / len(train_loader.dataset)
    ))

def test(epoch):
    model.eval()
    test_loss = 0
    for batch_idx, (data, _) in enumerate(test_loader):
        if cuda:
            data = Variable(data.cuda(), volatile=True)
        else:
            data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        test_loss += loss.data[0]
        
        # 最初のminibatchの入力画像と復元画像を保存
        if batch_idx == 0:
            n = 8
            comparison = torch.cat([data[:n],
                                    recon_batch.view(batch_size, 1, 28, 28)[:n]])
            save_image(comparison.data.cpu(),
                       '{}/reconstruction_{}.png'.format(out_dir, epoch), nrow=n)

- VAEは教師なし学習なのでラベルは使わない

In [43]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Epoch: 1 Average loss: 160.0181


KeyboardInterrupt: 